# Colab Setup

In [1]:
# Mount users Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Unzip the MOT16 Dataset if needed
!unzip -q "/content/drive/MyDrive/Deep_Learning/Final Project/MOT16.zip" -d "/content/drive/MyDrive/Deep_Learning/Final Project"

Mounted at /content/drive


In [2]:
# Installs
!pip install motmetrics -q
!pip install ultralytics -q
!pip install deep_sort_realtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 152.4 MB/s eta 0:00:00


In [3]:
# All Imports
import os
import sys
import cv2
import glob
import yaml
import torch
import shutil
import random
import configparser
#
import numpy as np
import pandas as pd
import motmetrics as mm
import albumentations as A
#
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
# Patch np.asfarray if it does not exist (NumPy 2.x) for motmetrics library
if not hasattr(np, "asfarray"):
    np.asfarray = lambda a: np.asarray(a, dtype=float)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Add path and folders
sys.path.append("/content/drive/MyDrive/Deep_Learning/Final Project/")
save_folder = "/content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO"
os.makedirs(save_folder, exist_ok=True)

In [5]:
# Configuration
# Seed
seed=42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Data
train_val_ratio=0.8
# Training Parameters
lr=1e-3
wd=5e-4
EPOCHS=15
BATCH_SIZE=2
# Validation and metrics
CONF_THRESHOLD = 0.7
IOU_THRESHOLD = 0.5
# Deepsort
max_age=10
n_init=3
max_cosine_distance=0.2


# YOLO Data Preprocessing

In [6]:
# Create Required YOLO Data
def create_yolo_labels(source_dir):
    for sequence_name in os.listdir(source_dir):
        sequence_dir = os.path.join(source_dir, sequence_name)

        # Skip if labels already exist
        labels_dir = os.path.join(sequence_dir, "labels")
        if os.path.exists(labels_dir):
            print(f"Skipping {sequence_name} (labels already exist)")
            continue

        # Read the sequence information
        seqinfo_path = os.path.join(sequence_dir, "seqinfo.ini")
        config = configparser.ConfigParser()
        config.read(seqinfo_path)
        imWidth = config.getint("Sequence","imWidth")
        imHeight = config.getint("Sequence","imHeight")

        # Create new YOLO labels folder
        new_labels = os.path.join(sequence_dir,"labels")
        os.makedirs(new_labels,exist_ok=True)

        # Ground Truth location
        gt_path = os.path.join(sequence_dir, "gt", "gt.txt")

        # Read ground truth using Pandas
        columns = ["frame", "id", "bb_left", "bb_top", "bb_width", "bb_height","conf", "x", "y", "z"]
        gt = pd.read_csv(gt_path, header=None, names=columns)

        # Filter rows for postive confidence
        filter_gt = gt[gt["conf"]>0].copy()

        # Calculate required label information for YOLO
        filter_gt["class_id"] = 0 # only one class pedestrians
        filter_gt["x_center"] = (filter_gt["bb_left"]+filter_gt["bb_width"]/2)/imWidth
        filter_gt["y_center"] = (filter_gt["bb_top"]+filter_gt["bb_height"]/2)/imHeight
        filter_gt["width_norm"] = filter_gt["bb_width"]/imWidth
        filter_gt["height_norm"] = filter_gt["bb_height"]/imHeight

        # Clip all values to [0,1] to fix any bounding issues
        for col in ["x_center", "y_center", "width_norm", "height_norm"]:
            filter_gt[col] = filter_gt[col].clip(0.0, 1.0)

        # Group ground truth per frame and save 1 file
        for frame_ids, group in filter_gt.groupby("frame"):
            labels_path=os.path.join(new_labels, f"{int(frame_ids):06d}.txt")
            group[["class_id", "x_center", "y_center", "width_norm", "height_norm"]].to_csv(labels_path, sep= " ", header=False, index=False, float_format="%.4f")

        print(f"Converted: {sequence_name}")

In [7]:
# Create YOLO File Structure
def train_val_split(source_dir, split_ratio, seed):
    sequences = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    sequences.sort()
    print("All sequences:", sequences)

    random.seed(seed)
    random.shuffle(sequences)

    split_idx = int(len(sequences) * split_ratio)
    train_sequence = sequences[:split_idx]
    val_sequence = sequences[split_idx:]

    print("Training sequences:", train_sequence)
    print("Validation sequences:", val_sequence)

    return train_sequence, val_sequence

def yolo_folder_structure(target_dir, source_dir, sequence_list, split_type):
    # Create empty folders
    os.makedirs(os.path.join(target_dir, "images", split_type), exist_ok=True)
    os.makedirs(os.path.join(target_dir, "labels", split_type), exist_ok=True)

    # Copy images and labels for YOLO folder
    for sequence in sequence_list:
        img_source = os.path.join(source_dir, sequence, "img1")
        lbl_source = os.path.join(source_dir, sequence, "labels")

        img_target = os.path.join(target_dir, "images", split_type, sequence)
        lbl_target = os.path.join(target_dir, "labels", split_type, sequence)

        os.makedirs(img_target, exist_ok=True)
        os.makedirs(lbl_target, exist_ok=True)

        # Copy images
        for f in os.listdir(img_source):
            shutil.copy(os.path.join(img_source, f), os.path.join(img_target, f))

        # Copy labels
        for f in os.listdir(lbl_source):
            shutil.copy(os.path.join(lbl_source, f), os.path.join(lbl_target, f))

    print(f"{split_type} sequences copied: {sequence_list}")

def create_yolo_data_yaml(data_dir, output_path="MOT16.yaml"):
    data_dict = {
        "train": os.path.join(data_dir, "images/train").replace("\\","/"),
        "val": os.path.join(data_dir, "images/val").replace("\\","/"),
        "nc": 1,
        "names": ["pedestrian"]
    }

    with open(output_path, "w") as f:
        yaml.dump(data_dict, f)

    print(f"YOLOv8 data.yaml saved as {output_path}")

In [10]:
# Prepare the data
create_yolo_labels(source_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train") # Create Labels
train_sequence, val_sequence = train_val_split(source_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train", split_ratio=train_val_ratio, seed=seed) # Split Data
yolo_folder_structure(target_dir="MOT16_YOLO", source_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train", sequence_list=train_sequence, split_type="train") # Store training data
yolo_folder_structure(target_dir="MOT16_YOLO", source_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train", sequence_list=val_sequence, split_type="val") # Store validation data
create_yolo_data_yaml(data_dir="MOT16_YOLO") # Create YAML file

Skipping MOT16-13 (labels already exist)
Skipping MOT16-09 (labels already exist)
Skipping MOT16-10 (labels already exist)
Skipping MOT16-11 (labels already exist)
Skipping MOT16-05 (labels already exist)
Skipping MOT16-02 (labels already exist)
Skipping MOT16-04 (labels already exist)
All sequences: ['MOT16-02', 'MOT16-04', 'MOT16-05', 'MOT16-09', 'MOT16-10', 'MOT16-11', 'MOT16-13']
Training sequences: ['MOT16-04', 'MOT16-09', 'MOT16-10', 'MOT16-05', 'MOT16-13']
Validation sequences: ['MOT16-02', 'MOT16-11']
train sequences copied: ['MOT16-04', 'MOT16-09', 'MOT16-10', 'MOT16-05', 'MOT16-13']
val sequences copied: ['MOT16-02', 'MOT16-11']
YOLOv8 data.yaml saved as MOT16.yaml


# Detector Training and Validation

In [11]:
# Training
model = YOLO("yolov8s.pt") # load model
results = model.train(data="MOT16.yaml", optimizer="AdamW", lr0=lr, weight_decay=wd, epochs = EPOCHS, batch=BATCH_SIZE, seed=seed, project = save_folder, # data augmentation and train the model
                      auto_augment=None, translate=.15, scale=0.25, fliplr=0.5, mosaic=0,
                      augmentations=[A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5), A.GaussianBlur(blur_limit=(3,5), sigma_limit=(0.1, 1.0), p=0.25)])

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, augmentations=[ColorJitter(p=0.5, brightness=(0.8, 1.2), contrast=(0.8, 1.2), hue=(-0.2, 0.2), saturation=(0.8, 1.2)), GaussianBlur(p=0.25, blur_limit=(3, 5), sigma_limit=(0.1, 1.0))], auto_augment=None, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=MOT16.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, m

In [12]:
# Evaluate the model
metrics = model.val(data="MOT16.yaml", conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, project = save_folder)
csv_string=metrics.to_csv()
csv_path = os.path.join(save_folder, "Evaluation_metrics.csv")
with open(csv_path, "w") as f:
    f.write(csv_string)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2775.3±1232.8 MB/s, size: 160.5 KB)
val: Scanning /content/MOT16_YOLO/labels/val/MOT16-02.cache... 1500 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1500/1500 3.4Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 94/94 10.9it/s 8.6s
                   all       1500      27007      0.994      0.224      0.609      0.471
Speed: 0.4ms preprocess, 1.1ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/val


# Detector Visualization

In [13]:
# Inference
def yolo_detections_to_txt(frames_folder, model_path, output_txt_path, conf, iou):
    model = YOLO(model_path)

    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith(".jpg")])

    # Open Detector txt file
    with open(output_txt_path, "w") as f_out:
        for frame_idx, frame_file in enumerate(frame_files, start=1):
            frame_path = os.path.join(frames_folder, frame_file)
            frame = cv2.imread(frame_path)

            # YOLO detections
            results = model.predict(frame, conf=conf, iou=iou, verbose=False)[0]
            for box, conf_tensor in zip(results.boxes.xyxy, results.boxes.conf): # xyxy = Boxes in pixel format [x1, y1, x2, y2]
                x1, y1, x2, y2 = box.tolist() # tensor to list and x1,y1 = top left corner and x2,y2 = bottom right corner
                w, h = x2-x1, y2-y1
                conf_val= float(conf_tensor)
                track_id=0 # placeholder ids because detection only

            # Write detections to Detector txt file
                f_out.write(f"{frame_idx},{track_id},{int(x1)},{int(y1)},{int(w)},{int(h)},{conf_val},-1,-1,-1\n")
    print(f"Detected results saved to {output_txt_path}")

In [14]:
# Visualization
def get_sequence_info(sequence_dir):
    seqinfo_path = os.path.join(sequence_dir, "seqinfo.ini")
    config = configparser.ConfigParser()
    config.read(seqinfo_path)
    fps = None
    width = None
    height = None
    fps = config.getint("Sequence", "frameRate", fallback=fps)
    width = config.getint("Sequence", "imWidth", fallback=width)
    height = config.getint("Sequence", "imHeight", fallback=height)
    return fps, (width, height)

def visualize_sequence(sequence_dir, results_file, output_video_path):
    # Load results
    results_df = pd.read_csv(results_file, header=None)
    results_df.columns = ["frame","id","x","y","w","h","conf","x3","y3","z3"]

    # Load frames
    image_paths = sorted(glob.glob(os.path.join(sequence_dir, "img1", "*.jpg")))

    # Video writer
    fps, frame_size = get_sequence_info(sequence_dir)
    first_img = cv2.imread(image_paths[0])
    frame_size = (first_img.shape[1], first_img.shape[0])
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, frame_size)

    # Assign random colors for IDs
    max_id = int(results_df["id"].max()) + 1
    np.random.seed(42)  # reproducible colors
    colors = np.random.randint(0, 255, size=(max_id, 3), dtype=np.uint8)

    # Process frames
    for img_path in image_paths:
        frame_num = int(os.path.splitext(os.path.basename(img_path))[0])
        img = cv2.imread(img_path)
        if img is None:
            continue

        frame_data = results_df[results_df["frame"] == frame_num]
        for _, row in frame_data.iterrows():
            track_id = int(row["id"])
            color = tuple(map(int, colors[track_id]))
            x, y, w_box, h_box = int(row["x"]), int(row["y"]), int(row["w"]), int(row["h"])
            cv2.rectangle(img, (x, y), (x + w_box, y + h_box), color, 2)
            cv2.putText(img, str(track_id), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        video_writer.write(img)
    video_writer.release()
    print(f"Video saved to {output_video_path}")

In [16]:
# Detect pedestrians
best_model = os.path.join(save_folder, "train/weights/best.pt")
det_output_path = os.path.join(save_folder, "Detect.txt")
yolo_detections_to_txt(frames_folder="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train/MOT16-02/img1/", model_path=best_model, output_txt_path=det_output_path, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD)

Detected results saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/Detect.txt


In [17]:
# Visualize
det_video_path = os.path.join(save_folder, "Detect.mp4")
visualize_sequence(sequence_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train/MOT16-02/", results_file=det_output_path, output_video_path=det_video_path)

Video saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/Detect.mp4


# ID Tracking

In [18]:
# Deepsort
def track_frames_to_mot_txt(frames_folder, model_path, output_txt_path, conf, iou, max_age, n_init, max_cosine_distance):
    # Load YOLOv8
    model = YOLO(model_path)
    tracker = DeepSort(
        max_age= max_age,                       # how long to keep a lost track
        n_init=n_init,                          # frames to confirm a track
        max_cosine_distance=max_cosine_distance # threshold for re-ID feature matching
    )

    # Sort frame files
    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith(".jpg")])

    # Open MOT txt file
    with open(output_txt_path, "w") as f_out:
        for frame_idx, frame_file in enumerate(frame_files, start=1):
            frame_path = os.path.join(frames_folder, frame_file)
            frame = cv2.imread(frame_path)

            # YOLO detections
            results = model.predict(frame, conf=conf, iou=iou, verbose=False, seed=42)[0]
            detections = []
            for box, conf_tensor in zip(results.boxes.xyxy, results.boxes.conf): # xyxy = Boxes in pixel format [x1, y1, x2, y2]
                x1, y1, x2, y2 = box.tolist() # tensor to list and x1,y1 = top left corner and x2,y2 = bottom right corner
                w, h = x2-x1, y2-y1
                detections.append(([x1, y1, w, h], float(conf_tensor))) # DeepSORT expects ([x, y, w, h], conf)

            # Update DeepSORT
            tracks = tracker.update_tracks(detections, frame=frame)

            # Write tracks to MOT txt
            for track in tracks:
                x, y, w, h = track.to_tlwh()  # top-left, width-height
                track_id = track.track_id
                conf_val = 1.0  # placeholder, tracker does not store confidence
                f_out.write(f"{frame_idx},{track_id},{int(x)},{int(y)},{int(w)},{int(h)},{conf_val},-1,-1,-1\n")

    print(f"Tracking results saved to {output_txt_path}")

In [19]:
# Tracking Metrics
def evaluate_mot_sequence(gt_txt_path, tracker_txt_path, csv_path, max_iou=0.5):
    # Load GT and tracker files
    gt = pd.read_csv(gt_txt_path, header=None)
    tr = pd.read_csv(tracker_txt_path, header=None)

    # Keep only first 7 columns: frame, id, x, y, w, h, conf
    gt = gt.iloc[:, :7]
    tr = tr.iloc[:, :7]

    gt.columns = ["frame","id","x","y","w","h","conf"]
    tr.columns = ["frame","id","x","y","w","h","conf"]

    gt = gt[gt["conf"] > 0].copy() # keep only evaluation boxes (conf > 0)
    tr = tr[tr["conf"] > 0].copy() # keep only evaluation boxes (conf > 0)

    # Initialize MOTAccumulator
    acc = mm.MOTAccumulator(auto_id=True)

    # Go frame by frame
    frames = sorted(gt["frame"].unique())
    for f in frames:
        gt_frame = gt[gt["frame"]==f]
        tr_frame = tr[tr["frame"]==f]

        gt_ids = gt_frame["id"].tolist()
        tr_ids = tr_frame["id"].tolist()

        gt_boxes = gt_frame[["x","y","w","h"]].values
        tr_boxes = tr_frame[["x","y","w","h"]].values

        # Compute IoU distance matrix
        distances = 1 - mm.distances.iou_matrix(gt_boxes, tr_boxes, max_iou=max_iou)
        acc.update(gt_ids, tr_ids, distances)

    # Create MetricsHost
    mh = mm.metrics.create()
    # Compute MOTChallenge metrics
    summary = mh.compute(acc, metrics=mm.metrics.motchallenge_metrics)
    summary.to_csv(csv_path)
    print(f"[OK] Metrics saved to {csv_path}")
    print(summary)

In [20]:
# Track id accross frames using DeepSort and save detections in MOT format
id_output_path = os.path.join(save_folder, "track.txt")
track_frames_to_mot_txt(frames_folder="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train/MOT16-02/img1/",model_path=best_model,output_txt_path=id_output_path, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, max_age=max_age, n_init=n_init, max_cosine_distance=max_cosine_distance)

Tracking results saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/track.txt


In [21]:
# Compute MOT metrics on DeepSort Result
metrics_output_path = os.path.join(save_folder, "MOT.csv")
evaluate_mot_sequence(gt_txt_path="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train/MOT16-02/gt/gt.txt", tracker_txt_path=id_output_path, csv_path=metrics_output_path)

[OK] Metrics saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/MOT.csv
       idf1       idp       idr    recall  precision  num_unique_objects  \
0  0.148486  0.556041  0.085684  0.150788    0.97853                  54   

   mostly_tracked  partially_tracked  mostly_lost  num_false_positives  \
0               4                 11           39                   59   

   num_misses  num_switches  num_fragmentations      mota      motp  \
0       15144            76                  75  0.143218  0.786879   

   num_transfer  num_ascend  num_migrate  
0            15          65            4  


In [22]:
# Tracking video
video_output_path = os.path.join(save_folder, "tracking_video.mp4")
visualize_sequence(sequence_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/train/MOT16-02/", results_file=id_output_path, output_video_path=video_output_path)

Video saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/tracking_video.mp4


# Testing video tracking

In [23]:
# Run Deepsort on testing video
testID_output_path = os.path.join(save_folder, "track_TEST3.txt")
track_frames_to_mot_txt(frames_folder="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/test/MOT16-03/img1/",model_path=best_model,output_txt_path=testID_output_path, conf=CONF_THRESHOLD, iou=IOU_THRESHOLD, max_age=max_age, n_init=n_init, max_cosine_distance=max_cosine_distance)

Tracking results saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/track_TEST3.txt


In [24]:
# Test Video with tracking
test_video_output_path = os.path.join(save_folder, "test_tracking_video_03.mp4")
visualize_sequence(sequence_dir="/content/drive/MyDrive/Deep_Learning/Final Project/MOT16/test/MOT16-03/", results_file=testID_output_path, output_video_path=test_video_output_path)

Video saved to /content/drive/MyDrive/Deep_Learning/Final Project/results_YOLO/test_tracking_video_03.mp4
